In [1]:
import pandas as pd
import numpy as np
from ofxparse import OfxParser
from datetime import datetime

In [2]:
# get downloaded statments
def get_files(filetype):
    files = []
    for f in os.listdir('statements/'):
        if not f.startswith('.') and ("." + filetype) in f: 
            files.append("statements/" + f)
    
    return files

In [3]:
# returnt he transaction from .ofx file type
def get_transactions(filename):
    with open(filename) as f:
        ofx = OfxParser.parse(f)

    account = ofx.account
    statement = account.statement
    transactions = statement.transactions

    return transactions

In [4]:
# build dataframe
df = pd.DataFrame(columns=['date', 'year', 'month', 'day', 'description', 'debit', 'credit'])

files = get_files("ofx")

for f in files:
    for t in get_transactions(f):
        debit = t.amount if t.type == "debit" else 0
        credit = t.amount if t.type == "credit" else 0
        new = {'date':t.date, 'year': t.date.year, 'month': t.date.month, 'day': t.date.day, 'description': t.memo, 'debit': debit, 'credit': credit}
        
        df = df.append(new, ignore_index=True)

df = df.astype({"description": str, "debit": float, "credit": float})

In [9]:
temp = pd.read_csv("zArchive/early18.csv")

In [12]:
temp.columns = [i.lower() for i in temp.columns]

In [32]:
order = [0,4,5,6,1,2,3]
temp = temp[[temp.columns[i] for i in order]]
temp.columns = df.columns

In [40]:
df = df.sort_values('date')

In [33]:
temp.to_pickle("merge.pkl")

In [43]:
x = pd.concat([temp, df], ignore_index=True)

In [54]:
temp.to_csv("18.csv")
df.to_csv("rest.csv")

In [62]:
x1 = pd.read_csv("18.csv")
x2 = pd.read_csv("rest.csv")

In [63]:
final = pd.concat([x1, x2], ignore_index=True)

In [67]:
final.to_pickle("transactions.pkl")